In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.set_option("display.max_columns", None)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("/kaggle/input/dont-overfit-ii/train.csv")

In [ ]:
test_df = pd.read_csv("/kaggle/input/dont-overfit-ii/test.csv")
sub = pd.read_csv

In [ ]:
train_df.head(10)

So it is a classification task

In [ ]:
print(len(test_df))
print()

In [ ]:
test_df.head(10)

In [ ]:
target = train_df["target"]
train_df = train_df.drop(["target"], axis = 1)

In [ ]:
Summary = pd.DataFrame(train_df.dtypes, columns=["dtype"])
Summary["max"] = train_df.max()
Summary["min"] = train_df.min()
Summary["null"] = train_df.isnull().sum()
Summary["first"] = train_df.iloc[0]
Summary["second"] = train_df.iloc[1]
Summary["third"] = train_df.iloc[2]
Summary

In [ ]:
train_df.isnull().sum().sum()
test_df.isnull().sum().sum()
# So, there are no null values

In [ ]:
target.value_counts()
# so, we can say that target value is not balanced

In [ ]:
train_df = train_df.drop(["id"], axis = 1)
test_df = test_df.drop(["id"], axis = 1)

In [ ]:
# train_df["max"] = train_df.max(axis=1)
# train_df["min"] = train_df.min(axis = 1)
# train_df["std"] = train_df.std(axis =1)
# train_df["skew"] = train_df.skew(axis =1)
# train_df["mean"] = train_df.mean(axis=1)
# train_df["median"] = train_df.median(axis=1)

In [ ]:
# test_df["max"] = test_df.max(axis=1)
# test_df["min"] = test_df.min(axis = 1)
# test_df["std"] = test_df.std(axis =1)
# test_df["skew"] = test_df.skew(axis =1)
# test_df["mean"] = test_df.mean(axis=1)
# test_df["median"] = test_df.median(axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import mutual_info_classif

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(train_df.drop('target',axis=1), target, stratify = target,
#                                                     random_state = 123, test_size = 0.25) 
X_train, X_test, y_train, y_test = train_test_split(train_df, target,stratify = target,
                                                    random_state = 123, test_size = 0.25) 

In [ ]:
mi = mutual_info_classif(X_train, y_train, random_state=123)

In [ ]:
selectedFeature  =  np.array( mi > 0.02)
print('number of selected columns',selectedFeature.sum())
print('number of selected columns',train_df.drop('target',axis =1).columns[selectedFeature])

In [ ]:
dropFeaturesdrop = train_df.drop('target', axis =1).columns[~selectedFeature]
X_train.drop(dropFeaturesdrop,axis= 1,inplace= True)
X_test.drop(dropFeaturesdrop,axis= 1 ,inplace= True)
test_df.drop(dropFeaturesdrop,axis= 1 ,inplace= True)
# y_train.drop(dropFeaturesdrop,axis= 1 ,inplace= True)
# y_test.drop(dropFeaturesdrop,axis= 1 ,inplace= True)

In [ ]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)
test_df = scale.transform(test_df)

In [ ]:
# Grid search cross validation
grid={"C":np.logspace(-4,4,7),
      "penalty":["l1","l2"]
     }# l1 lasso l2 ridge
lm = LogisticRegression()
logreg_cv=GridSearchCV(lm, grid, cv = 10, scoring='roc_auc' )
logreg_cv.fit(X_train,y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("roc_auc :",logreg_cv.best_score_)

In [ ]:

# lm = LogisticRegression(C = 0.0001, penalty = 'l2' )
lm = LogisticRegression(C = 0.1, class_weight = 'balanced', penalty =  'l2')
lm.fit(X_train,y_train)
# lm.fit(X_train, y_train)

In [ ]:
y_pred = lm.predict_proba(X_test)
y_pred = y_pred[:,1]

In [ ]:
# tps, fps, _ = roc_curve(X_test, y_pred)
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(auc(fpr, tpr))
# Score: 0.6253196930946292, 0.46 pretty bad
# Score with regularization is 0.5537084398976982
# above with added features: 0.5588235294117647
# with just regularization: 0.7429667519181585

In [ ]:
y_res = lm.predict_proba(test_df)
y_res = y_res[:,1]

In [ ]:
sub = pd.read_csv("/kaggle/input/dont-overfit-ii/sample_submission.csv")
print(len(sub))
sub["target"] = y_res
sub.to_csv("lr_reg_lib_linear.csv", index=False)



In [ ]:
len(test_df)